# Определение стоимости автомобилей

## Подготовка данных

Импорт необходимых бибилиотек

In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
import time
import warnings
warnings.filterwarnings('ignore')

Загрузка данных

In [2]:
try:
    df = pd.read_csv('/datasets/autos.csv')
except:
    df = pd.read_csv('autos.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Найдем дубликаты

In [5]:
df.duplicated().sum()

4

Избавимся от дубликатов

In [6]:
df.drop_duplicates(inplace=True)

Преобразуем формат даты в столбце `DateCreated` для дальнейшего использования

In [7]:
df['DateCreated'] = pd.DatetimeIndex(df['DateCreated']).year

Избавимся столбцов, не содержащих полезных данных для моделей, а также столбца `NumberOfPictures` где все значения нулевые

In [8]:
df.drop(['DateCrawled', 'LastSeen', 'NumberOfPictures', 'PostalCode'], axis=1, inplace=True)

Посмотрим на долю пропущенных значений в датасете

In [9]:
df.isna().mean()

Price                0.000000
VehicleType          0.105795
RegistrationYear     0.000000
Gearbox              0.055968
Power                0.000000
Model                0.055607
Kilometer            0.000000
RegistrationMonth    0.000000
FuelType             0.092828
Brand                0.000000
NotRepaired          0.200793
DateCreated          0.000000
dtype: float64

Заполним недостающие данные значением `unknown`

In [10]:
df.fillna({x:'unknown' for x in ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired']}, inplace=True)

Посмотрим на разброс данных

In [11]:
df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,DateCreated
count,354365.000000,354365.000000,354365.000000,354365.000000,354365.000000,354365.000000
mean,4416.679830,2004.234481,110.093816,128211.363989,5.714650,2015.999924
std,4514.176349,90.228466,189.851330,37905.083858,3.726432,0.009046
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,2014.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,2016.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,2016.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,2016.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,2016.000000


Избавимся от машин с ничтожной стоимостью

In [12]:
df = df.query(' Price > 10 ')

Избавимся от автомобилей, произведенных до начала производства автомобилей и тех, что произведены до года включения в базу

In [13]:
df = df.query('RegistrationYear > 1885 and (RegistrationYear <= DateCreated)')

Теперь можно удалить столбец `DateCreated`

In [14]:
df.drop(['DateCreated'], axis=1, inplace=True)

Избавимся от автомобилей, которые не смогут передвигаться без посторонней помощи и тех, которые мощнее самого мощного авто в мире

In [15]:
df = df.query(' Power > 10 and Power < 2500')

In [16]:
df.reset_index(drop=True, inplace=True)

In [18]:
df.drop(['RegistrationMonth'], axis=1, inplace=True)

## Обучение моделей

Разделим данные на обучающую, валидационную и тестовую выборки в соотношении 60%:20%:20%

In [19]:
features = df.drop('Price', axis=1)
target = df['Price']

cat_features = ['VehicleType','Gearbox','Model','FuelType','Brand', 'NotRepaired']

features_train, features_valid, target_train, target_valid = (train_test_split(features, target, test_size=0.40,
                                                                              random_state=12345))

features_valid, features_test, target_valid, target_test = (train_test_split(features_valid, target_valid,
                                                                             test_size=0.50, random_state=12345))

In [20]:
display(features_train.shape[0]/features.shape[0])
display(features_valid.shape[0]/features.shape[0])
display(features_test.shape[0]/features.shape[0])

0.6

0.2

0.2

### CatBoost

In [21]:
model_cat = CatBoostRegressor(random_state=2019, learning_rate=0.5, n_estimators=200)

start_time = time.time()
model_cat.fit(features_train, target_train, silent=True,
              eval_set=(features_valid, target_valid), cat_features=cat_features)

end_time = time.time()
learn_time_cat = round(end_time-start_time, 2)

In [22]:
start_time = time.time()
predict_cat = model_cat.predict(features_test)
end_time = time.time()
predict_time_cat = round(end_time-start_time, 2)
rmse_cat = mean_squared_error(target_test, predict_cat)**0.5

### LightGBM

In [25]:
features_train[cat_features] = features_train[cat_features].astype('category')
features_valid[cat_features] = features_valid[cat_features].astype('category')
features_test[cat_features] = features_test[cat_features].astype('category')

In [26]:
model_lgbm = LGBMRegressor(n_estimators=200, random_state=2019, learning_rate=0.5, objective='rmse')

In [27]:
start_time = time.time()
model_lgbm.fit(features_train, target_train, eval_set=(features_valid, target_valid), verbose=0)
end_time = time.time()
learn_time_lgbm = round(end_time-start_time, 2)

In [28]:
start_time = time.time()
predict_lgbm=model_lgbm.predict(features_test)
end_time = time.time()
predict_time_lgbm = round(end_time-start_time, 2)
rmse_lgbm = mean_squared_error(target_test, predict_lgbm)**0.5

### RandomForestRegressor

Применим кодирование one-hot для преобразования категоральных признаков

In [29]:
df_ohe = pd.get_dummies(df, drop_first=True)

In [34]:
features_ohe = df_ohe.drop('Price', axis=1)
target_ohe = df_ohe['Price']

features_train_ohe, features_valid_ohe, target_train_ohe, target_valid_ohe = train_test_split(
    features_ohe, target_ohe, test_size=0.4, random_state=12345)

features_valid_ohe, features_test_ohe, target_valid_ohe, target_test_ohe = (train_test_split(features_valid_ohe, target_valid_ohe,
                                                                             test_size=0.50, random_state=12345))

In [35]:
display(features_train_ohe.shape[0]/features_ohe.shape[0])
display(features_valid_ohe.shape[0]/features_ohe.shape[0])
display(features_test_ohe.shape[0]/features_ohe.shape[0])

0.6

0.2

0.2

In [36]:
model_forest = RandomForestRegressor(random_state=2019, n_estimators=200)
start_time = time.time()
model_forest.fit(features_train_ohe, target_train_ohe)
end_time = time.time()
learn_time_forest = round(end_time-start_time, 2)

In [37]:
start_time = time.time()
predict_forest = model_forest.predict(features_test_ohe)
end_time = time.time()
predict_time_forest = round(end_time-start_time, 2)
rmse_forest = mean_squared_error(target_test_ohe, predict_forest)**0.5

### DummyRegressor

In [38]:
dummy_median = DummyRegressor(strategy="median")
start_time = time.time()
dummy_median.fit(features_train_ohe, target_train_ohe)
end_time = time.time()
learn_time_dummy = round(end_time-start_time, 2)
start_time = time.time()
predict_dummy = dummy_median.predict(features_test_ohe)
end_time = time.time()
predict_time_dummy = round(end_time-start_time, 2)
rmse_dummy = mean_squared_error(target_test_ohe, predict_dummy)**0.5

## Анализ моделей

In [39]:
pd.DataFrame({'модель': ['CatBoost', 'LightGBM', 'RandomForest', 'DummyRegressor'], 
              'время обучения (c)': [learn_time_cat, learn_time_lgbm, learn_time_forest, learn_time_dummy],
              'время предсказания (c)': [predict_time_cat, predict_time_lgbm, predict_time_forest, predict_time_dummy],
              'RMSE': [rmse_cat, rmse_lgbm, rmse_forest, rmse_dummy]})

,модель,время обучения (c),время предсказания (c),RMSE
0,CatBoost,7.97,0.07,1592.291528
1,LightGBM,0.59,0.08,1569.520296
2,RandomForest,405.51,3.85,1569.233297
3,DummyRegressor,0.01,0.00,4909.504343


Как видно из таблицы модель **LightGBM** лидирует по времени обучения и лишь немного уступает по времени предсказания модели CatBoost. Модели CatBoost, LightGBM и RandomForest обладают примерно равным качеством предсказания (по метрике RMSE), значительно лучшим, чем модель DummyRegressor на основе медианных значений.

Заказчику можно рекомедовать к использованию модель **LightGBM**